All filenames could be changed if you have different ones

# Generate simulated data from an additive model

## 1. Prepare raw data.

## 2. Split the individuals in the fam file into three parts.

In [ ]:
import pandas as pd
import random

fam =       #the path and the name of fam file
n1 =        #the number of individuals to be put into the training set
n2 =        #the number of individuals to be put into the validation set
n3 =        #the number of individuals to be put into the test set

fam = pd.read_csv(fam,sep=' ',header=None)
set1 = list(fam.iloc[:,0].values)
fam1 = random.sample(set1,n1)
set2 = list(set(set1).difference(set(fam1)))
fam2 = random.sample(set2,n2)
fam3 = random.sample(list(set(set2).difference(set(fam2))),n3)
fam1 = pd.DataFrame(fam1)
fam2 = pd.DataFrame(fam2)
fam3 = pd.DataFrame(fam3)
fam1[1] = fam1.iloc[:,0].values
fam2[1] = fam2.iloc[:,0].values
fam3[1] = fam3.iloc[:,0].values
fam1.to_csv('train.txt',sep=' ',header=False,index=False)
fam2.to_csv('valid.txt',sep=' ',header=False,index=False)
fam3.to_csv('test.txt',sep=' ',header=False,index=False)

## 3. Split raw genotype.

In [ ]:
import os
os.system('plink --bfile raw_data --keep train.txt --make-bed --out training_set')
os.system('plink --bfile raw_data --keep valid.txt --make-bed --out validation_set')
os.system('plink --bfile raw_data --keep test.txt --make-bed --out test_set')

## 4. Select a list of risk SNPs and assign their effect sizes.

In [ ]:
import pandas as pd
import random
import numpy as np
bim = pd.read_csv('bim file',sep='\t',header=None)

# here we have an example of 10000 risk SNPs with 2500 small effect sizes, 5000 moderate effect sizes 
# and 2500 strong effect sizes
n1 = 10000
n2 = 2500
n3 = 5000
n4 = 2500
snp = random.sample(list(bim.iloc[:,1].values),n1)
eff1 = np.random.normal(0,0.1,n2)
eff2 = np.random.normal(0,0.1**0.5,n3)
eff3 = np.random.normal(0,1,n4)
eff = np.concatenate((eff1,eff2,eff3))
snp = pd.DataFrame(snp)
snp[1] = list(eff)
snp.to_csv('causal.snplist',header=False,index=False,sep=' ')

## 5. Apply GCTA to generate the phenotypes for the individuals in the training set, validation set and test set.

In [ ]:
import os
num_cases_train = 
num_controls_train = 
num_cases_valid = 
num_controls_valid =
num_cases_test = 
num_controls_test =
heritability = 
prevalence =
num_replicates =
os.system('gcta64 --bfile training_set --simu-cc {} {} --simu-causal-loci causal.snplist --simu-hsq {} --simu-k {} --simu-rep {} --out output'.format(num_cases_train,num_controls_train,heritability,prevalence,num_replicates))
os.system('gcta64 --bfile validation_set --simu-cc {} {} --simu-causal-loci causal.snplist --simu-hsq {} --simu-k {} --simu-rep {} --out output'.format(num_cases_valid,num_controls_valid,heritability,prevalence,num_replicates))
os.system('gcta64 --bfile test_set --simu-cc {} {} --simu-causal-loci causal.snplist --simu-hsq {} --simu-k {} --simu-rep {} --out output'.format(num_cases_test,num_controls_test,heritability,prevalence,num_replicates))

## 6. Generate the association statistics by PLINK on the training set.

In [ ]:
import os

os.system('plink --bfile training_set --logistic --out output') # other filtering parameters could be added (e.g. --maf 0.01)

# Generate simulated data from a nonlinear model

## 0. Use HapGen2 to generate genotype data (you could skip this part if you already have the target genotype data)

In [ ]:
import os

os.system('hapgen2 -m .map_file -l .legend_file -h .haps_file -o output -n number_of_controls number_of_cases -Ne effective_population_size -dl physical_location  risk_allele heterozygote_disease_risk homozygote_disease_risk -no_haps_output')

## 1. Prepare genotype data

## 2. Use SIMER to generate quantitative phenotypes

In [ ]:
import os

os.system('Rscript generate_phenotypes.R')

## 3. Convert quantitative phenotypes into binary phenotypes and generate training, validation and test set

In [ ]:
import random
import pandas as pd
import os
import sys
import numpy as np

def to_binary_phenotype(path,outpath,popu,chro):
    random.seed(10)
    df_fam = pd.read_csv('{}{}_chr{}.fam'.format(path,popu,chro),sep=' ',header=None)
    boundary = np.quantile(df_fam.iloc[:,5].values,0.8) # 0.8 here means that we set the disease prevalence to be 0.2
    index1 = df_fam.index[df_fam.iloc[:,5]>=boundary]
    index2 = df_fam.index[df_fam.iloc[:,5]<boundary]
    if len(index1) != 0.2*len(df_fam.index):
        sys.exit('The number of cases is not correct')
    df_fam.loc[index1,5] = 2
    df_fam.loc[index2,5] = 1
    df_fam[5] = df_fam[5].astype(int)
    df_fam.to_csv('{}{}_chr{}.fam'.format(path,popu,chro),sep=' ',header=False,index=False)
    n1 = 10000 # number of cases/controls in the training set
    n2 = 5000 # number of cases/controls in the validation set
    n3 = 3000 # number of cases/controls in the testing set
    total_case = list(df_fam.loc[index1,0].values)
    total_control = list(df_fam.loc[index2,0].values)

    training_case = random.sample(total_case,n1)
    total_case_left = list(set(total_case)-set(training_case))
    valid_case = random.sample(total_case_left,n2)
    test_case = list(set(total_case_left)-set(valid_case))

    training_control = random.sample(total_control,n1)
    total_control_left1 = list(set(total_control)-set(training_control))
    valid_control = random.sample(total_control_left1,n2)
    total_control_left2 = list(set(total_control_left1)-set(valid_control))
    test_control = random.sample(total_control_left2,n3)

    training = training_case+training_control
    valid = valid_case+valid_control
    testing = test_case+test_control
    training = sorted(training)
    valid = sorted(valid)
    testing = sorted(testing)

    df_train = pd.DataFrame(training)
    df_train[1] = training
    df_valid = pd.DataFrame(valid)
    df_valid[1] = valid
    df_test = pd.DataFrame(testing)
    df_test[1] = testing

    df_train.to_csv('{}{}_chr{}_training_id.txt'.format(path,popu,chro),sep=' ',header=False,index=False)
    df_valid.to_csv('{}{}_chr{}_valid_id.txt'.format(path,popu,chro),sep=' ',header=False,index=False)
    df_test.to_csv('{}{}_chr{}_test_id.txt'.format(path,popu,chro),sep=' ',header=False,index=False)

    os.system('plink --bfile {}{}_chr{} --maf 0.01 --make-bed --out {}{}_chr{}_filtered'.format(path,popu,chro,path,popu,chro))
    os.system('plink --bfile {}{}_chr{}_filtered --keep {}{}_chr{}_training_id.txt --make-bed --out {}{}_chr{}_training'.format(path,popu,chro,path,popu,chro,outpath,popu,chro))
    os.system('plink --bfile {}{}_chr{}_filtered --keep {}{}_chr{}_valid_id.txt --make-bed --out {}{}_chr{}_valid'.format(path,popu,chro,path,popu,chro,outpath,popu,chro))
    os.system('plink --bfile {}{}_chr{}_filtered --keep {}{}_chr{}_test_id.txt --make-bed --out {}{}_chr{}_test'.format(path,popu,chro,path,popu,chro,outpath,popu,chro))
    

# Summary statistics

## Some methods could use summary statistics below, others have required formats as shown in their documentations.
## LDpred2 could use the summary statistics generated by the script below.

In [ ]:
import pandas as pd
diseases=['1002','1348','1066','1111','1134','1154','1220','1242','1265','1294','1297','1374','1065','1068','1113','1136','1207','1224','1243','1293','1295','1452']
for phe in diseases:
    if phe == '1002' or phe == '1348' or phe == '1207':
        logi = pd.read_csv('./{}/{}_train.assoc.logistic'.format(phe,phe),sep=' +')
        set1 = list(range(0,len(logi.index),12)) # here 12 is related to the number of factors in the covariance file. The goal is to only include 'additive' one, 
        # and skip others considering only one factor
        logi1 = logi.iloc[set1,:]
        logi1.index=list(range(len(logi1.index)))
        logi1.to_csv('./{}/{}_train_v2.assoc.logistic'.format(phe,phe),sep=' ',index=False)
    else:
        logi = pd.read_csv('./{}/{}_train.assoc.logistic'.format(phe,phe),sep=' +')
        set1 = list(range(0,len(logi.index),13)) # we have one more factor 'sex' here, so it is 13.
        logi1 = logi.iloc[set1,:]
        logi1.index=list(range(len(logi1.index)))
        logi1.to_csv('./{}/{}_train_v2.assoc.logistic'.format(phe,phe),sep=' ',index=False)

## Tweedie, tlpSum, elastSum, lassosum, EB-PRS could use the summary statistics generated by the script below.

In [ ]:
import pandas as pd
diseases=['1002','1348','1066','1111','1134','1154','1220','1242','1265','1294','1297','1374','1065','1068','1113','1136','1207','1224','1243','1293','1295','1452']
for phe in diseases:
    logi = pd.read_csv('./{}/{}_train_v2.assoc.logistic'.format(phe,phe),sep=' +')
    frq = pd.read_csv('./{}/{}_train.frq'.format(phe,phe),sep=' +')
    set1=[]
    for j in range(len(frq.index)):
        if frq.loc[j,'SNP'] not in logi.loc[:,'SNP'].values:
            set1.append(j)
    frq.drop(set1,axis=0,inplace=True)
    frq.index = range(len(frq.index))
    if all(logi.loc[:,'SNP'].values == frq.loc[:,'SNP'].values):
            logi['A2'] = frq.loc[:,'A2'].values
            logi['MAF'] = frq.loc[:,'MAF'].values
            name=['CHR','SNP','BP','A1','A2','STAT','P','TEST','NMISS','OR','SE','L95','U95','MAF']
            logi=logi[name]
            logi.to_csv('./{}/{}_train_v3.assoc.logistic'.format(phe,phe),sep=' ',index=False)

# Machine learning models

## 1. Generate SNP list with p-values < 5E-3, < 5E-4, < 5E-5, < 5E-6

In [ ]:
from collections import defaultdict

def generate(index,hsq):
    print(index)
    valid=defaultdict(float)
    test=defaultdict(float)
    infile1=open('../train_rep{}_{}.assoc.logistic.adjusted'.format(index,hsq))
    snplist1=open('./SNPselection/'+index+'_5E3_SNP_{}'.format(hsq),'w')
    snplist2=open('./SNPselection/'+index+'_5E4_SNP_{}'.format(hsq),'w')
    snplist3=open('./SNPselection/'+index+'_5E5_SNP_{}'.format(hsq),'w')
    snplist4=open('./SNPselection/'+index+'_5E6_SNP_{}'.format(hsq),'w')

    SNP1=0
    SNP2=0
    SNP3=0
    SNP4=0
    
    linenum=0
    for line in infile1:
        if linenum>0:
            A=line.strip('\n').rsplit()
            if float(A[3])<0.005:
                snplist1.write(A[1]+'\n')
                SNP1+=1
            if float(A[3])<0.0005:
                snplist2.write(A[1]+'\n')
                SNP2+=1
            if float(A[3])<0.00005:
                snplist3.write(A[1]+'\n')
                SNP3+=1
            if float(A[3])<0.000005:
                snplist4.write(A[1]+'\n')
                SNP4+=1
        linenum+=1
    snpvalidate.write(str(index)+' '+str(SNP1)+' '+str(SNP2)+' '+str(SNP3)+' '+str(SNP4)+'\n')
    infile1.close()
    snplist1.close()
    snplist2.close()
    snplist3.close()
    snplist4.close()
for hsq in [0.5]:
    snpvalidate=open('./SNPselection/stat_{}'.format(hsq),'w')
    for index in range(1,11):
        #print(index)
        generate(str(index),str(hsq))
    snpvalidate.close()

## 2. Extract SNPs satisfying p-values < 5E-3, < 5E-4, < 5E-5, < 5E-6 and remove individuals with phenotypes of -9

In [ ]:
import os
def generate(index,hsq):
    os.system('plink --bfile ../train_merge_white_Britich_clean_rep'+index+'_'+hsq+' --prune --extract ./SNPselection/'+index+'_5E6_SNP_'+hsq+' --make-bed --out ./SNPextraction/'+index+'_5E6_train_extract_'+hsq+'\n')
    os.system('plink --bfile ../train_merge_white_Britich_clean_rep'+index+'_'+hsq+' --prune --extract ./SNPselection/'+index+'_5E5_SNP_'+hsq+' --make-bed --out ./SNPextraction/'+index+'_5E5_train_extract_'+hsq+'\n')
    os.system('plink --bfile ../train_merge_white_Britich_clean_rep'+index+'_'+hsq+' --prune --extract ./SNPselection/'+index+'_5E4_SNP_'+hsq+' --make-bed --out ./SNPextraction/'+index+'_5E4_train_extract_'+hsq+'\n')
    os.system('plink --bfile ../train_merge_white_Britich_clean_rep'+index+'_'+hsq+' --prune --extract ./SNPselection/'+index+'_5E3_SNP_'+hsq+' --make-bed --out ./SNPextraction/'+index+'_5E3_train_extract_'+hsq+'\n')
    
    os.system('plink --bfile ../vali_merge_white_Britich_clean_'+index+'_'+hsq+' --prune --extract ./SNPselection/'+index+'_5E6_SNP_'+hsq+' --make-bed --out ./SNPextraction/'+index+'_5E6_valid_extract_'+hsq+'\n')
    os.system('plink --bfile ../vali_merge_white_Britich_clean_'+index+'_'+hsq+' --prune --extract ./SNPselection/'+index+'_5E5_SNP_'+hsq+' --make-bed --out ./SNPextraction/'+index+'_5E5_valid_extract_'+hsq+'\n')
    os.system('plink --bfile ../vali_merge_white_Britich_clean_'+index+'_'+hsq+' --prune --extract ./SNPselection/'+index+'_5E4_SNP_'+hsq+' --make-bed --out ./SNPextraction/'+index+'_5E4_valid_extract_'+hsq+'\n')
    os.system('plink --bfile ../vali_merge_white_Britich_clean_'+index+'_'+hsq+' --prune --extract ./SNPselection/'+index+'_5E3_SNP_'+hsq+' --make-bed --out ./SNPextraction/'+index+'_5E3_valid_extract_'+hsq+'\n')
    
    os.system('plink --bfile ../test_merge_white_Britich_clean_'+index+'_'+hsq+' --prune --extract ./SNPselection/'+index+'_5E6_SNP_'+hsq+' --make-bed --out ./SNPextraction/'+index+'_5E6_test_extract_'+hsq+'\n')
    os.system('plink --bfile ../test_merge_white_Britich_clean_'+index+'_'+hsq+' --prune --extract ./SNPselection/'+index+'_5E5_SNP_'+hsq+' --make-bed --out ./SNPextraction/'+index+'_5E5_test_extract_'+hsq+'\n')
    os.system('plink --bfile ../test_merge_white_Britich_clean_'+index+'_'+hsq+' --prune --extract ./SNPselection/'+index+'_5E4_SNP_'+hsq+' --make-bed --out ./SNPextraction/'+index+'_5E4_test_extract_'+hsq+'\n')
    os.system('plink --bfile ../test_merge_white_Britich_clean_'+index+'_'+hsq+' --prune --extract ./SNPselection/'+index+'_5E3_SNP_'+hsq+' --make-bed --out ./SNPextraction/'+index+'_5E3_test_extract_'+hsq+'\n')
    

for hsq in [0.5]:
    for index in range(1,11):
        generate(str(index),str(hsq))

## 3. LD pruning

In [ ]:
import os
def generate(index,hsq):
    print(index)
    os.system('plink --bfile ./SNPextraction/'+index+'_5E6_train_extract_'+hsq+' --indep-pairwise 50 5 0.5 --out ./LDpruning/train_5E6_'+index+'_'+hsq+'\n')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E5_train_extract_'+hsq+' --indep-pairwise 50 5 0.5 --out ./LDpruning/train_5E5_'+index+'_'+hsq+'\n')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E4_train_extract_'+hsq+' --indep-pairwise 50 5 0.5 --out ./LDpruning/train_5E4_'+index+'_'+hsq+'\n')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E3_train_extract_'+hsq+' --indep-pairwise 50 5 0.5 --out ./LDpruning/train_5E3_'+index+'_'+hsq+'\n')

    os.system('plink --bfile ./SNPextraction/'+index+'_5E6_valid_extract_'+hsq+' --indep-pairwise 50 5 0.5 --out ./LDpruning/valid_'+index+'_'+hsq+'\n')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E5_valid_extract_'+hsq+' --indep-pairwise 50 5 0.5 --out ./LDpruning/valid_'+index+'_'+hsq+'\n')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E4_valid_extract_'+hsq+' --indep-pairwise 50 5 0.5 --out ./LDpruning/valid_'+index+'_'+hsq+'\n')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E3_valid_extract_'+hsq+' --indep-pairwise 50 5 0.5 --out ./LDpruning/valid_'+index+'_'+hsq+'\n')

    os.system('plink --bfile ./SNPextraction/'+index+'_5E6_test_extract_'+hsq+' --indep-pairwise 50 5 0.5 --out ./LDpruning/test_'+index+'_'+hsq+'\n')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E5_test_extract_'+hsq+' --indep-pairwise 50 5 0.5 --out ./LDpruning/test_'+index+'_'+hsq+'\n')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E4_test_extract_'+hsq+' --indep-pairwise 50 5 0.5 --out ./LDpruning/test_'+index+'_'+hsq+'\n')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E3_test_extract_'+hsq+' --indep-pairwise 50 5 0.5 --out ./LDpruning/test_'+index+'_'+hsq+'\n')


for hsq in [0.5]:
    for index in range(1,11):
        generate(str(index),str(hsq))

## 4. Extract SNPs that are low LD

In [ ]:
import os

def generate(index,hsq):
    print(index)
    os.system('plink --bfile ./SNPextraction/'+index+'_5E6_train_extract_'+hsq+' --extract ./LDpruning/train_5E6_'+index+'_'+hsq+'.prune.in --make-bed --out ./final/'+index+'_'+hsq+'_train_5E6_SNP')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E5_train_extract_'+hsq+' --extract ./LDpruning/train_5E5_'+index+'_'+hsq+'.prune.in --make-bed --out ./final/'+index+'_'+hsq+'_train_5E5_SNP')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E4_train_extract_'+hsq+' --extract ./LDpruning/train_5E4_'+index+'_'+hsq+'.prune.in --make-bed --out ./final/'+index+'_'+hsq+'_train_5E4_SNP')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E3_train_extract_'+hsq+' --extract ./LDpruning/train_5E3_'+index+'_'+hsq+'.prune.in --make-bed --out ./final/'+index+'_'+hsq+'_train_5E3_SNP')

    os.system('plink --bfile ./final/'+index+'_'+hsq+'_train_5E6_SNP --recodeA --out ./final/'+index+'_'+hsq+'_train_5E6_recodeA')
    os.system('plink --bfile ./final/'+index+'_'+hsq+'_train_5E5_SNP --recodeA --out ./final/'+index+'_'+hsq+'_train_5E5_recodeA')
    os.system('plink --bfile ./final/'+index+'_'+hsq+'_train_5E4_SNP --recodeA --out ./final/'+index+'_'+hsq+'_train_5E4_recodeA')
    os.system('plink --bfile ./final/'+index+'_'+hsq+'_train_5E3_SNP --recodeA --out ./final/'+index+'_'+hsq+'_train_5E3_recodeA')


    os.system('plink --bfile ./SNPextraction/'+index+'_5E6_valid_extract_'+hsq+' --extract ./LDpruning/train_5E6_'+index+'_'+hsq+'.prune.in --make-bed --out ./final/'+index+'_'+hsq+'_valid_5E6_SNP')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E5_valid_extract_'+hsq+' --extract ./LDpruning/train_5E5_'+index+'_'+hsq+'.prune.in --make-bed --out ./final/'+index+'_'+hsq+'_valid_5E5_SNP')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E4_valid_extract_'+hsq+' --extract ./LDpruning/train_5E4_'+index+'_'+hsq+'.prune.in --make-bed --out ./final/'+index+'_'+hsq+'_valid_5E4_SNP')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E3_valid_extract_'+hsq+' --extract ./LDpruning/train_5E3_'+index+'_'+hsq+'.prune.in --make-bed --out ./final/'+index+'_'+hsq+'_valid_5E3_SNP')

    os.system('plink --bfile ./final/'+index+'_'+hsq+'_valid_5E6_SNP --recodeA --out ./final/'+index+'_'+hsq+'_valid_5E6_recodeA')
    os.system('plink --bfile ./final/'+index+'_'+hsq+'_valid_5E5_SNP --recodeA --out ./final/'+index+'_'+hsq+'_valid_5E5_recodeA')
    os.system('plink --bfile ./final/'+index+'_'+hsq+'_valid_5E4_SNP --recodeA --out ./final/'+index+'_'+hsq+'_valid_5E4_recodeA')
    os.system('plink --bfile ./final/'+index+'_'+hsq+'_valid_5E3_SNP --recodeA --out ./final/'+index+'_'+hsq+'_valid_5E3_recodeA')


    os.system('plink --bfile ./SNPextraction/'+index+'_5E6_test_extract_'+hsq+' --extract ./LDpruning/train_5E6_'+index+'_'+hsq+'.prune.in --make-bed --out ./final/'+index+'_'+hsq+'_test_5E6_SNP')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E5_test_extract_'+hsq+' --extract ./LDpruning/train_5E5_'+index+'_'+hsq+'.prune.in --make-bed --out ./final/'+index+'_'+hsq+'_test_5E5_SNP')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E4_test_extract_'+hsq+' --extract ./LDpruning/train_5E4_'+index+'_'+hsq+'.prune.in --make-bed --out ./final/'+index+'_'+hsq+'_test_5E4_SNP')
    os.system('plink --bfile ./SNPextraction/'+index+'_5E3_test_extract_'+hsq+' --extract ./LDpruning/train_5E3_'+index+'_'+hsq+'.prune.in --make-bed --out ./final/'+index+'_'+hsq+'_test_5E3_SNP')

    os.system('plink --bfile ./final/'+index+'_'+hsq+'_test_5E6_SNP --recodeA --out ./final/'+index+'_'+hsq+'_test_5E6_recodeA')
    os.system('plink --bfile ./final/'+index+'_'+hsq+'_test_5E5_SNP --recodeA --out ./final/'+index+'_'+hsq+'_test_5E5_recodeA')
    os.system('plink --bfile ./final/'+index+'_'+hsq+'_test_5E4_SNP --recodeA --out ./final/'+index+'_'+hsq+'_test_5E4_recodeA')
    os.system('plink --bfile ./final/'+index+'_'+hsq+'_test_5E3_SNP --recodeA --out ./final/'+index+'_'+hsq+'_test_5E3_recodeA')


for hsq in [0.5]:
    for index in range(1,11):
        generate(str(index),str(hsq))


## 5. Generate .npy files

In [ ]:
from collections import defaultdict
import matplotlib
import os
import re
import numpy as np
import operator
from scipy import stats as sta
import math
import multiprocessing
import operator
import scipy
from itertools import groupby
from operator import itemgetter 
def generate_G_P(path_data):
    data=np.genfromtxt(path_data,dtype=str)
    SNPindexlist=[]
    SNP_id=[]
    all_id=[]
    #indindexlist=[]
    ind_genotype=[]
    all_disease=[]
    totalind=data.shape[0]
    #print(totalind)
    for i in range(totalind):
        #print(i)
        if i==0:
            snpnum=data[0].shape[0]
            for j in range(6,snpnum):
                SNPindexlist.append(j)
                SNP_id.append(data[0][j].split('_')[0])
        else:
            if len(SNPindexlist)==1:
                tmp=data[i][6]
                if tmp=='NA':
                    tmp='0'
            else:
                tmp=list(itemgetter(*SNPindexlist)(data[i]))
                for x in range(len(tmp)):
                    if tmp[x]=='NA':
                        tmp[x]='0'
            #indindexlist.append(indall.index(data[i][0]))
            ind_genotype.append(list(map(int,tmp)))
            all_id.append(data[i][0])
            all_disease.append(int(data[i][5])-1)
    all_genotype=sta.zscore(ind_genotype)
    #print(all_genotype[0:5])
    return all_genotype,all_disease,all_id,SNP_id


def Cal_score(index,hsq,pv):
    #print(index)
    outpath1='./train/'
    outpath2='./valid/'
    outpath3='./test/'

    out1_geno=outpath1+str(index)+'_'+hsq+'_5E{}_genotype_train'.format(pv)
    out2_geno=outpath2+str(index)+'_'+hsq+'_5E{}_genotype_valid'.format(pv)
    out3_geno=outpath3+str(index)+'_'+hsq+'_5E{}_genotype_test'.format(pv)


    out1_disease=outpath1+str(index)+'_'+hsq+'_disease_train'
    out2_disease=outpath2+str(index)+'_'+hsq+'_disease_valid'
    out3_disease=outpath3+str(index)+'_'+hsq+'_disease_test'
    
    out1_id=outpath1+str(index)+'_'+hsq+'_indid_train'
    out2_id=outpath2+str(index)+'_'+hsq+'_indid_valid'
    out3_id=outpath3+str(index)+'_'+hsq+'_indid_test'
    
    out1_SNP = outpath1+str(index)+'_'+hsq+'_5E{}_SNP_train'.format(pv)
    out2_SNP = outpath2+str(index)+'_'+hsq+'_5E{}_SNP_valid'.format(pv)
    out3_SNP = outpath3+str(index)+'_'+hsq+'_5E{}_SNP_test'.format(pv)
        
    [all_genotype_1,all_disease_1,all_id_1,SNP_id_1]=generate_G_P(dir_path+index+'_'+hsq+'_train_5E{}_recodeA.raw'.format(pv))
    [all_genotype_2,all_disease_2,all_id_2,SNP_id_2]=generate_G_P(dir_path+index+'_'+hsq+'_valid_5E{}_recodeA.raw'.format(pv))
    [all_genotype_3,all_disease_3,all_id_3,SNP_id_3]=generate_G_P(dir_path+index+'_'+hsq+'_test_5E{}_recodeA.raw'.format(pv))
  

    np.save(out1_geno, all_genotype_1)
    np.save(out2_geno, all_genotype_2)
    np.save(out3_geno, all_genotype_3)


    np.save(out1_disease,all_disease_1)    
    np.save(out2_disease,all_disease_2)
    np.save(out3_disease,all_disease_3)

    np.save(out1_id,all_id_1)
    np.save(out2_id,all_id_2)
    np.save(out3_id,all_id_3)

    np.save(out1_SNP,SNP_id_1)
    np.save(out2_SNP,SNP_id_2)
    np.save(out3_SNP,SNP_id_3)

#indall=[]
#fam_path = os.path.abspath('..')
#infile1=open('{}/sim_chr1.fam'.format(fam_path))
#for line in infile1:
#    A=line.strip('\n').split()
#    indall.append(A[0])
#infile1.close()


dir_path='./final/'
pool = multiprocessing.Pool(10)
for index in range(1,11):
    for hsq in [0.5]:
        for pv in [3,4,5,6]:
            pool.apply_async(Cal_score,(str(index),str(hsq),pv,))
pool.close()
pool.join()

## 6. adaboost

In [ ]:
from collections import defaultdict
import matplotlib
import os
import numpy as np
import pandas as pd
import operator
from scipy import stats as sta
import math
import operator
from itertools import groupby
from sklearn.metrics import roc_auc_score
from operator import itemgetter
from sklearn.linear_model import LogisticRegression
from scipy.stats.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from scipy.stats import norm
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import multiprocessing
import pickle
from sklearn.metrics import average_precision_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
dir_path1='./train/'
dir_path2='./valid/'
dir_path3='./test/'

def predict(index):
    global auc
    global name
    global ap
    #print(index)
    disease_train=np.load(dir_path1+index+'_disease_train.npy')
    #disease_valid=np.load(dir_path2+index+'_'+hsq+'_disease_valid.npy')
    #disease_test=np.load(dir_path3+index+'_'+hsq+'_disease_test.npy')

    Cxlist=[DecisionTreeClassifier(),LogisticRegression(),ExtraTreeClassifier(),GaussianNB()]
    set1=[]
    for s in [1,2,3,4]:
        #print(s)
        valid=0
        if s==1 and os.path.exists(dir_path1+index+'_5E3_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E3_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E3_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E3_genotype_test.npy')
            valid=1
        elif s==2 and os.path.exists(dir_path1+index+'_5E4_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E4_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E4_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E4_genotype_test.npy')
            valid=1
        elif s==3 and os.path.exists(dir_path1+index+'_5E5_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E5_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E5_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E5_genotype_test.npy')
            valid=1
        elif s==4 and os.path.exists(dir_path1+index+'_5E6_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E6_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E6_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E6_genotype_test.npy')
            valid=1
        if valid==1:
            for t in [1,2,3,4]:
                Cx=Cxlist[t-1]
                ada = AdaBoostClassifier(base_estimator=Cx)
                ada.fit(Geno_train,disease_train)
                Y=ada.predict_proba(Geno_valid)
                np.save('./ada/'+index+'_'+str(s)+'_'+str(t)+'_valid.npy',Y)
                disease_valid=pd.read_csv('../{}/{}_valid_new.fam'.format(index,index),sep=' +',header=None)
                list1=[]
                for i in range(len(disease_valid.index)):
                    if disease_valid.iloc[i,5]==-9:
                        list1.append(i)
                disease_valid.drop(list1,axis=0,inplace=True)
                disease_valid.index=list(range(len(disease_valid.index)))
                set2=[]
                for j in range(len(disease_valid.index)):
                    set2.append(Y[j][1])
                set3 = [int(x)-1 for x in disease_valid.iloc[:,5].values]
                set1.append(roc_auc_score(set3,set2))
    max_index=set1.index(max(set1))
    s1=int((max_index)/4)+1
    Cx1=Cxlist[max_index-(s1-1)*4]
    name.append('{}_s{}_t{}'.format(index,s1,Cx1))
    Geno_train=np.load(dir_path1+index+'_5E{}_genotype_train.npy'.format(s1+2))
    Geno_test=np.load(dir_path3+index+'_5E{}_genotype_test.npy'.format(s1+2))
    ada1 = AdaBoostClassifier(base_estimator=Cx1)
    ada1.fit(Geno_train,disease_train)
    Y1=ada1.predict_proba(Geno_test)
    np.save('./ada/'+index+'_'+str(s)+'_'+str(t)+'_test.npy',Y1)
    disease_test=pd.read_csv('../{}/{}_test_new.fam'.format(index,index),sep=' +',header=None)
    list2=[]
    for k1 in range(len(disease_test.index)):
        if disease_test.iloc[k1,5]==-9:
            list2.append(k1)
    disease_test.drop(list2,axis=0,inplace=True)
    disease_test.index=list(range(len(disease_test.index)))
    set4=[]
    for k2 in range(len(disease_test.index)):
        set4.append(Y1[k2][1])
    set5 = [int(x)-1 for x in disease_test.iloc[:,5].values]
    auc.append(roc_auc_score(set5,set4))
    ap.append(average_precision_score(set5,set4))
    df_score = pd.DataFrame(disease_test.iloc[:,0].values)
    df_score[1] = set4
    df_score[2] = set5
    df_score.to_csv('./ada/test_predi_score_{}_ada.txt'.format(index),sep=' ',index=False,header=False)
 
name=[]
auc=[]
ap=[]
#pool = multiprocessing.Pool(4)

diseases=['1002','1348','1066','1111','1134','1154','1220','1242','1265','1294','1297','1374','1065','1068','1113','1136','1207','1224','1243','1293','1295','1452']

for index in diseases:
    #print(m)
    predict(index)
    #pool.apply_async(predict,(index,))
#pool.close()
#pool.join()

df = pd.DataFrame(name)
df[1] = auc
df.to_csv('test_auc_results_ada.txt',sep='\t',header=False,index=False)
df1 = pd.DataFrame(name)
df1[1] = ap
df1.to_csv('test_ap_results_ada.txt',sep='\t',header=False,index=False)

## 7. gradient boosting

In [ ]:
from collections import defaultdict
import matplotlib
import os
import pandas as pd
import numpy as np
import operator
from scipy import stats as sta
import math
import operator
from itertools import groupby
from sklearn.metrics import roc_auc_score
from operator import itemgetter
from sklearn.linear_model import LogisticRegression
from scipy.stats.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from scipy.stats import normfrom collections import defaultdict
import matplotlib
from sklearn.metrics import average_precision_score
import os
import numpy as np
import operator
from scipy import stats as sta
import math
import pandas as pd
import operator
from itertools import groupby
from sklearn.metrics import roc_auc_score
from operator import itemgetter
from sklearn.linear_model import LogisticRegression
from scipy.stats.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from scipy.stats import norm
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import multiprocessing
import pickle
from sklearn.ensemble import RandomForestClassifier

dir_path1='./train/'
dir_path2='./valid/'
dir_path3='./test/'

def predict(index):
    #print(index)
    global name
    global auc
    global ap
    disease_train=np.load(dir_path1+index+'_disease_train.npy')
    Cxlist=[0.0001,0.001,0.01,0.1]
    set1=[]
    for s in [1,2,3,4]:
        print(s)
        valid=0
        if s==1 and os.path.exists(dir_path1+index+'_5E3_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E3_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E3_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E3_genotype_test.npy')
            valid=1
        elif s==2 and os.path.exists(dir_path1+index+'_5E4_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E4_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E4_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E4_genotype_test.npy')
            valid=1
        elif s==3 and os.path.exists(dir_path1+index+'_5E5_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E5_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E5_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E5_genotype_test.npy')
            valid=1
        elif s==4 and os.path.exists(dir_path1+index+'_5E6_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E6_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E6_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E6_genotype_test.npy')
            valid=1
        if valid==1:
            for t in [1,2,3,4]:
                Cx=Cxlist[t-1]
                RF = RandomForestClassifier(min_impurity_decrease=Cx)
                RF.fit(Geno_train,disease_train)
                Y=RF.predict_proba(Geno_valid)
                #print(Y)
                np.save('./RF/'+index+'_'+str(s)+'_'+str(t)+'_valid.npy',Y)
                disease_valid=pd.read_csv('../{}/{}_valid_new.fam'.format(index,index),sep=' +',header=None)
                list1=[]
                for i in range(len(disease_valid.index)):
                    if disease_valid.iloc[i,5]==-9:
                        list1.append(i)
                disease_valid.drop(list1,axis=0,inplace=True)
                disease_valid.index=list(range(len(disease_valid.index)))
                set2=[]
                for j in range(len(disease_valid.index)):
                    set2.append(Y[j][1])
                set3 = [int(x)-1 for x in disease_valid.iloc[:,5].values]
                set1.append(roc_auc_score(set3,set2))
    max_index=set1.index(max(set1))
    s1=int((max_index)/4)+1
    Cx1=Cxlist[max_index-(s1-1)*4]
    name.append('{}_s{}_t{}'.format(index,s1,Cx1))
    Geno_train=np.load(dir_path1+index+'_5E{}_genotype_train.npy'.format(s1+2))
    Geno_test=np.load(dir_path3+index+'_5E{}_genotype_test.npy'.format(s1+2))
    RF1=RandomForestClassifier(min_impurity_decrease=Cx1)
    RF1.fit(Geno_train,disease_train)
    Y1=RF1.predict_proba(Geno_test)
    #print(Y1)
    np.save('./RF/'+index+'_'+str(s)+'_'+str(t)+'_test.npy',Y1)
    disease_test=pd.read_csv('../{}/{}_test_new.fam'.format(index,index),sep=' +',header=None)
    list2=[]
    for k1 in range(len(disease_test.index)):
        if disease_test.iloc[k1,5]==-9:
            list2.append(k1)
    disease_test.drop(list2,axis=0,inplace=True)
    disease_test.index=list(range(len(disease_test.index)))
    set4=[]
    for k2 in range(len(disease_test.index)):
        set4.append(Y1[k2][1])
    set5 = [int(x)-1 for x in disease_test.iloc[:,5].values]
    auc.append(roc_auc_score(set5,set4))
    ap.append(average_precision_score(set5,set4))
    df_score = pd.DataFrame(disease_test.iloc[:,0].values)
    df_score[1] = set4
    df_score[2] = set5
    df_score.to_csv('./RF/test_predi_score_{}_RF.txt'.format(index),sep=' ',index=False,header=False)

name=[]
auc=[] 
ap=[]
#pool = multiprocessing.Pool(4)

diseases=['1002','1348','1066','1111','1134','1154','1220','1242','1265','1294','1297','1374','1065','1068','1113','1136','1207','1224','1243','1293','1295','1452']
for index in diseases:
    #print(m)
    predict(index)
    #pool.apply_async(predict,(index,))
#pool.close()
#pool.join()

df = pd.DataFrame(name)
df[1] = auc
df.to_csv('test_auc_results_RF.txt',sep='\t',header=False,index=False)
df1 = pd.DataFrame(name)
df1[1] = ap
df1.to_csv('test_ap_results_RF.txt',sep='\t',header=False,index=False)

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import multiprocessing
import pickle
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

dir_path1='./train/'
dir_path2='./valid/'
dir_path3='./test/'


def predict(index):
    #print(index)
    global auc
    global name
    global ap
    disease_train=np.load(dir_path1+index+'_disease_train.npy')
    Cxlist=[0.0001,0.001,0.01,0.1]
    set1=[]
    for s in [1,2,3,4]:
        #print(s)
        valid=0
        if s==1 and os.path.exists(dir_path1+index+'_5E3_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E3_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E3_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E3_genotype_test.npy')
            valid=1
        elif s==2 and os.path.exists(dir_path1+index+'_5E4_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E4_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E4_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E4_genotype_test.npy')
            valid=1
        elif s==3 and os.path.exists(dir_path1+index+'_5E5_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E5_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E5_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E5_genotype_test.npy')
            valid=1
        elif s==4 and os.path.exists(dir_path1+index+'_5E6_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E6_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E6_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E6_genotype_test.npy')
            valid=1
        if valid==1:
            for t in [1,2,3,4]:
                Cx=Cxlist[t-1]
                GB = GradientBoostingClassifier(min_impurity_decrease=Cx)
                GB.fit(Geno_train,disease_train)
                Y=GB.predict_proba(Geno_valid)
                np.save('./GB/'+index+'_'+str(s)+'_'+str(t)+'_valid.npy',Y)
                disease_valid=pd.read_csv('../{}/{}_valid_new.fam'.format(index,index),sep=' +',header=None)
                list1=[]
                for i in range(len(disease_valid.index)):
                    if disease_valid.iloc[i,5]==-9:
                        list1.append(i)
                disease_valid.drop(list1,axis=0,inplace=True)
                disease_valid.index=list(range(len(disease_valid.index)))
                set2=[]
                for j in range(len(disease_valid.index)):
                    set2.append(Y[j][1])
                set3 = [int(x)-1 for x in disease_valid.iloc[:,5].values]
                set1.append(roc_auc_score(set3,set2))
    max_index=set1.index(max(set1))
    s1=int((max_index)/4)+1
    Cx1=Cxlist[max_index-(s1-1)*4]
    name.append('{}_s{}_t{}'.format(index,s1,Cx1))
    Geno_train=np.load(dir_path1+index+'_5E{}_genotype_train.npy'.format(s1+2))
    #Geno_valid=np.load(dir_path2+index+'_'+hsq+'_5E{}_genotype_valid.npy'.format(s1+2))
    Geno_test=np.load(dir_path3+index+'_5E{}_genotype_test.npy'.format(s1+2))
    GB1 =GradientBoostingClassifier(min_impurity_decrease=Cx1)
    GB1.fit(Geno_train,disease_train)
    Y1=GB1.predict_proba(Geno_test)
    np.save('./GB/'+index+'_'+str(s)+'_'+str(t)+'_test.npy',Y1)
    disease_test=pd.read_csv('../{}/{}_test_new.fam'.format(index,index),sep=' +',header=None)
    list2=[]
    for k1 in range(len(disease_test.index)):
        if disease_test.iloc[k1,5]==-9:
            list2.append(k1)
    disease_test.drop(list2,axis=0,inplace=True)
    disease_test.index=list(range(len(disease_test.index)))
    set4=[]
    for k2 in range(len(disease_test.index)):
        set4.append(Y1[k2][1])
    set5 = [int(x)-1 for x in disease_test.iloc[:,5].values]
    auc.append(roc_auc_score(set5,set4))
    ap.append(average_precision_score(set5,set4))
    df_score = pd.DataFrame(disease_test.iloc[:,0].values)
    df_score[1] = set4
    df_score[2] = set5
    df_score.to_csv('./GB/test_predi_score_{}_GB.txt'.format(index),sep=' ',index=False,header=False)
    #print('index:'+str(index))
    #print('set1:'+str(len(set1)))
    #print('auc:'+str(len(auc)))
    #print('name:'+str(len(name)))
    #print('done')
name=[]
auc=[]
ap=[]
#pool = multiprocessing.Pool(4)

diseases=['1002','1348','1066','1111','1134','1154','1220','1242','1265','1294','1297','1374','1065','1068','1113','1136','1207','1224','1243','1293','1295','1452']
for index in diseases:
    #print(m)
    predict(index)
    #pool.apply_async(predict,(index,))
#pool.close()
#pool.join()

df = pd.DataFrame(name)
df[1] = auc
df.to_csv('test_auc_results_GB.txt',sep='\t',header=False,index=False)
df1 = pd.DataFrame(name)
df1[1] = ap
df1.to_csv('test_ap_results_GB.txt',sep='\t',header=False,index=False)

## 8. lasso regression

In [ ]:
from collections import defaultdict
import matplotlib
from sklearn.metrics import average_precision_score
import os
import numpy as np
import operator
from scipy import stats as sta
import math
import operator
from itertools import groupby
from sklearn.metrics import roc_auc_score
from operator import itemgetter
from sklearn.linear_model import LogisticRegression
from scipy.stats.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from scipy.stats import norm
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import multiprocessing
import pandas as pd
import pickle
from  sklearn.neural_network import MLPClassifier

dir_path1='./train/'
dir_path2='./valid/'
dir_path3='./test/'
def predict(index):
    #print(index)
    global auc
    global name
    global ap
    disease_train=np.load(dir_path1+index+'_disease_train.npy')
    Cxlist=[0.0001,0.001,0.01,0.1]
    set1=[]
    for s in [1,2,3,4]:
        #print(s)
        valid=0
        print(dir_path1+index+'_5E3_genotype_train.npy')
        if s==1 and os.path.exists(dir_path1+index+'_5E3_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E3_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E3_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E3_genotype_test.npy')
            valid=1
        elif s==2 and os.path.exists(dir_path1+index+'_5E4_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E4_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E4_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E4_genotype_test.npy')
            valid=1
        elif s==3 and os.path.exists(dir_path1+index+'_5E5_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E5_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E5_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E5_genotype_test.npy')
            valid=1
        elif s==4 and os.path.exists(dir_path1+index+'_5E6_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E6_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E6_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E6_genotype_test.npy')
            valid=1
        if valid==1:
            for t in [1,2,3,4]:
                Cx=Cxlist[t-1]
                LR = LogisticRegression(penalty='l1', C=Cx,solver='liblinear', max_iter=10000)
                LR.fit(Geno_train,disease_train)
                Y=LR.predict_proba(Geno_valid)
                np.save('./LR/'+index+'_'+str(s)+'_'+str(t)+'_valid.npy',Y)
                disease_valid=pd.read_csv('../{}/{}_valid_new.fam'.format(index,index),sep=' +',header=None)
                list1=[]
                for i in range(len(disease_valid.index)):
                    if disease_valid.iloc[i,5]==-9:
                        list1.append(i)
                disease_valid.drop(list1,axis=0,inplace=True)
                disease_valid.index=list(range(len(disease_valid.index)))
                set2=[]
                for j in range(len(disease_valid.index)):
                    set2.append(Y[j][1])
                set3 = [int(x)-1 for x in disease_valid.iloc[:,5].values]
                set1.append(roc_auc_score(set3,set2))
    print(len(set1))
    max_index=set1.index(max(set1))
    s1 = int((max_index)/4)+1
    Cx1=Cxlist[max_index-(s1-1)*4]
    name.append('{}_s{}_t{}'.format(index,s1,Cx1))
    Geno_train=np.load(dir_path1+index+'_5E{}_genotype_train.npy'.format(s1+2))
    Geno_test=np.load(dir_path3+index+'_5E{}_genotype_test.npy'.format(s1+2))
    LR1 = LogisticRegression(penalty='l1', C=Cx1,solver='liblinear', max_iter=10000)
    LR1.fit(Geno_train,disease_train)
    Y1=LR1.predict_proba(Geno_test)
    np.save('./LR/'+index+'_'+str(s)+'_'+str(t)+'_test.npy',Y1)
    disease_test=pd.read_csv('../{}/{}_test_new.fam'.format(index,index),sep=' +',header=None)
    list2=[]
    for k1 in range(len(disease_test.index)):
        if disease_test.iloc[k1,5]==-9:
            list2.append(k1)
    disease_test.drop(list2,axis=0,inplace=True)
    disease_test.index=list(range(len(disease_test.index)))
    set4=[]
    for k2 in range(len(disease_test.index)):
        set4.append(Y1[k2][1])
    set5 = [int(x)-1 for x in disease_test.iloc[:,5].values]
    auc.append(roc_auc_score(set5,set4))
    ap.append(average_precision_score(set5,set4))
    df_score = pd.DataFrame(disease_test.iloc[:,0].values)
    df_score[1] = set4
    df_score[2] = set5
    df_score.to_csv('./LR/test_predi_score_{}_LR.txt'.format(index),sep=' ',index=False,header=False)

name=[]
auc=[]
ap=[]
#pool = multiprocessing.Pool(4)

diseases=['1002','1348','1066','1111','1134','1154','1220','1242','1265','1294','1297','1374','1065','1068','1113','1136','1207','1224','1243','1293','1295','1452']
for index in diseases:
    #print(m)
    predict(index)
    #pool.apply_async(predict,(index,))
#pool.close()
#pool.join()

df = pd.DataFrame(name)
df[1] = auc
df.to_csv('test_auc_results_LR.txt',sep='\t',header=False,index=False)
df1 = pd.DataFrame(name)
df1[1] = ap
df1.to_csv('test_ap_results_LR.txt',sep='\t',header=False,index=False)

## 9. neural network

In [ ]:
from collections import defaultdict
import matplotlib
import os
import numpy as np
import operator
from sklearn.metrics import average_precision_score
from scipy import stats as sta
import math
import operator
from itertools import groupby
from sklearn.metrics import roc_auc_score
from operator import itemgetter
from sklearn.linear_model import LogisticRegression
from scipy.stats.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from scipy.stats import norm
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import multiprocessing
import pickle
import pandas as pd
from  sklearn.neural_network import MLPClassifier

dir_path1='./train/'
dir_path2='./valid/'
dir_path3='./test/'


def predict(index):
    #print(index)
    global name
    global auc
    global ap
    disease_train=np.load(dir_path1+index+'_disease_train.npy')
    Cxlist=[(30,30),(10,10,10,10),(20,20,20,20),(5,5,5,5)]
    set1=[]
    for s in [1,2,3,4]:
        #print(s)
        valid=0
        if s==1 and os.path.exists(dir_path1+index+'_5E3_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E3_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E3_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E3_genotype_test.npy')
            valid=1
        elif s==2 and os.path.exists(dir_path1+index+'_5E4_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E4_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E4_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E4_genotype_test.npy')
            valid=1
        elif s==3 and os.path.exists(dir_path1+index+'_5E5_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E5_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E5_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E5_genotype_test.npy')
            valid=1
        elif s==4 and os.path.exists(dir_path1+index+'_5E6_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E6_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E6_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E6_genotype_test.npy')
            valid=1
        if valid==1:
            for t in [1,2,3,4]:
                Cx=Cxlist[t-1]
                NN=MLPClassifier(hidden_layer_sizes=Cx,max_iter=1000)
                NN.fit(Geno_train,disease_train)
                Y=NN.predict_proba(Geno_valid)
                np.save('./NN/'+index+'_'+str(s)+'_'+str(t)+'_valid.npy',Y)
                disease_valid=pd.read_csv('../{}/{}_valid_new.fam'.format(index,index),sep=' +',header=None)
                list1=[]
                for i in range(len(disease_valid.index)):
                    if disease_valid.iloc[i,5]==-9:
                        list1.append(i)
                disease_valid.drop(list1,axis=0,inplace=True)
                disease_valid.index=list(range(len(disease_valid.index)))
                set2=[]
                for j in range(len(disease_valid.index)):
                    set2.append(Y[j][1])
                set3 = [int(x)-1 for x in disease_valid.iloc[:,5].values]
                set1.append(roc_auc_score(set3,set2))
    max_index=set1.index(max(set1))
    #print('len(set1):'+str(len(set1)))
    s1=int((max_index)/4)+1
    Cx1=Cxlist[max_index-(s1-1)*4]
    name.append('{}_s{}_t{}'.format(index,s1,Cx1))
    Geno_train=np.load(dir_path1+index+'_5E{}_genotype_train.npy'.format(s1+2))
    Geno_test=np.load(dir_path3+index+'_5E{}_genotype_test.npy'.format(s1+2))
    #print('name:'+str(len(name)))
    NN1=MLPClassifier(hidden_layer_sizes=Cx1,max_iter=1000)
    NN1.fit(Geno_train,disease_train)
    Y1=NN1.predict_proba(Geno_test)
    np.save('./NN/'+index+'_'+str(s)+'_'+str(t)+'_test.npy',Y1)
    disease_test=pd.read_csv('../{}/{}_test_new.fam'.format(index,index),sep=' +',header=None)
    list2=[]
    for k1 in range(len(disease_test.index)):
        if disease_test.iloc[k1,5]==-9:
            list2.append(k1)
    disease_test.drop(list2,axis=0,inplace=True)
    disease_test.index=list(range(len(disease_test.index)))
    set4=[]
    for k2 in range(len(disease_test.index)):
        set4.append(Y1[k2][1])
    set5 = [int(x)-1 for x in disease_test.iloc[:,5].values]
    #print('len(set5):'+str(len(set5)))
    auc.append(roc_auc_score(set5,set4))
    ap.append(average_precision_score(set5,set4))
    df_score = pd.DataFrame(disease_test.iloc[:,0].values)
    df_score[1] = set4
    df_score[2] = set5
    df_score.to_csv('./NN/test_predi_score_{}_NN.txt'.format(index),sep=' ',index=False,header=False)

name=[]
auc=[]
ap=[]
#pool = multiprocessing.Pool(5)

diseases=['1002','1348','1066','1111','1134','1154','1220','1242','1265','1294','1297','1374','1065','1068','1113','1136','1207','1224','1243','1293','1295','1452']
for index in diseases:
    #print(m)
    predict(index)
    #pool.apply_async(predict,(index,))
#pool.close()
#pool.join()
df = pd.DataFrame(name)
df[1] = auc
df.to_csv('test_auc_results_NN.txt',sep='\t',header=False,index=False)
df1 = pd.DataFrame(name)
df1[1] = ap
df1.to_csv('test_ap_results_NN.txt',sep='\t',header=False,index=False)

## 10. random forest

In [ ]:
from collections import defaultdict
import matplotlib
from sklearn.metrics import average_precision_score
import os
import numpy as np
import operator
from scipy import stats as sta
import math
import pandas as pd
import operator
from itertools import groupby
from sklearn.metrics import roc_auc_score
from operator import itemgetter
from sklearn.linear_model import LogisticRegression
from scipy.stats.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from scipy.stats import norm
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import multiprocessing
import pickle
from sklearn.ensemble import RandomForestClassifier

dir_path1='./train/'
dir_path2='./valid/'
dir_path3='./test/'

def predict(index):
    #print(index)
    global name
    global auc
    global ap
    disease_train=np.load(dir_path1+index+'_disease_train.npy')
    Cxlist=[0.0001,0.001,0.01,0.1]
    set1=[]
    for s in [1,2,3,4]:
        print(s)
        valid=0
        if s==1 and os.path.exists(dir_path1+index+'_5E3_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E3_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E3_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E3_genotype_test.npy')
            valid=1
        elif s==2 and os.path.exists(dir_path1+index+'_5E4_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E4_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E4_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E4_genotype_test.npy')
            valid=1
        elif s==3 and os.path.exists(dir_path1+index+'_5E5_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E5_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E5_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E5_genotype_test.npy')
            valid=1
        elif s==4 and os.path.exists(dir_path1+index+'_5E6_genotype_train.npy'):
            Geno_train=np.load(dir_path1+index+'_5E6_genotype_train.npy')
            Geno_valid=np.load(dir_path2+index+'_5E6_genotype_valid.npy')
            Geno_test=np.load(dir_path3+index+'_5E6_genotype_test.npy')
            valid=1
        if valid==1:
            for t in [1,2,3,4]:
                Cx=Cxlist[t-1]
                RF = RandomForestClassifier(min_impurity_decrease=Cx)
                RF.fit(Geno_train,disease_train)
                Y=RF.predict_proba(Geno_valid)
                #print(Y)
                np.save('./RF/'+index+'_'+str(s)+'_'+str(t)+'_valid.npy',Y)
                disease_valid=pd.read_csv('../{}/{}_valid_new.fam'.format(index,index),sep=' +',header=None)
                list1=[]
                for i in range(len(disease_valid.index)):
                    if disease_valid.iloc[i,5]==-9:
                        list1.append(i)
                disease_valid.drop(list1,axis=0,inplace=True)
                disease_valid.index=list(range(len(disease_valid.index)))
                set2=[]
                for j in range(len(disease_valid.index)):
                    set2.append(Y[j][1])
                set3 = [int(x)-1 for x in disease_valid.iloc[:,5].values]
                set1.append(roc_auc_score(set3,set2))
    max_index=set1.index(max(set1))
    s1=int((max_index)/4)+1
    Cx1=Cxlist[max_index-(s1-1)*4]
    name.append('{}_s{}_t{}'.format(index,s1,Cx1))
    Geno_train=np.load(dir_path1+index+'_5E{}_genotype_train.npy'.format(s1+2))
    Geno_test=np.load(dir_path3+index+'_5E{}_genotype_test.npy'.format(s1+2))
    RF1=RandomForestClassifier(min_impurity_decrease=Cx1)
    RF1.fit(Geno_train,disease_train)
    Y1=RF1.predict_proba(Geno_test)
    #print(Y1)
    np.save('./RF/'+index+'_'+str(s)+'_'+str(t)+'_test.npy',Y1)
    disease_test=pd.read_csv('../{}/{}_test_new.fam'.format(index,index),sep=' +',header=None)
    list2=[]
    for k1 in range(len(disease_test.index)):
        if disease_test.iloc[k1,5]==-9:
            list2.append(k1)
    disease_test.drop(list2,axis=0,inplace=True)
    disease_test.index=list(range(len(disease_test.index)))
    set4=[]
    for k2 in range(len(disease_test.index)):
        set4.append(Y1[k2][1])
    set5 = [int(x)-1 for x in disease_test.iloc[:,5].values]
    auc.append(roc_auc_score(set5,set4))
    ap.append(average_precision_score(set5,set4))
    df_score = pd.DataFrame(disease_test.iloc[:,0].values)
    df_score[1] = set4
    df_score[2] = set5
    df_score.to_csv('./RF/test_predi_score_{}_RF.txt'.format(index),sep=' ',index=False,header=False)

name=[]
auc=[] 
ap=[]
#pool = multiprocessing.Pool(4)

diseases=['1002','1348','1066','1111','1134','1154','1220','1242','1265','1294','1297','1374','1065','1068','1113','1136','1207','1224','1243','1293','1295','1452']
for index in diseases:
    #print(m)
    predict(index)
    #pool.apply_async(predict,(index,))
#pool.close()
#pool.join()

df = pd.DataFrame(name)
df[1] = auc
df.to_csv('test_auc_results_RF.txt',sep='\t',header=False,index=False)
df1 = pd.DataFrame(name)
df1[1] = ap
df1.to_csv('test_ap_results_RF.txt',sep='\t',header=False,index=False)

# Ensemble method

## In the script below, 'new_best_vali_auc_{}.txt' is a file consists of one row and two columns. A row stands for a disease, and the first column is a string consists of the disease name and the best tuning parameters of methods ('best' means highest AUROC), they are linked by '_' (e.g. diabetes_1_0.5, diabetes is the disease name, 1 and 0.5 are the two tuning parameters for a method), and the second column is AUROC calculated by a method for a disease. '.profile' file is the output file of PLINK when calculating PRS. 'new_score*.profile' is the PRS calculated on the validation set. 'new_test_score*.profile' is the PRS calculated on the test set. The suffix of files containing PRS of PRSice and PRSice2 is '.best' but not '.profile'.

In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

methods = ['PRScs','LDpred2','P+T','PRSice','DBSLMM','Tweedie','SBLUP','SbayesR','PRSice2','lassosum','EBPRS','tlpSum','elastSum']

for me in methods:
    if me == 'tlpSum' or me == 'elastSum':
        path = './penRegSum/'
        exec("df_{} = pd.read_csv('{}new_best_vali_auc_{}.txt',sep='\t',header=None,engine='python')".format(me,path,me))
        exec('auc_{} = df_{}[1].to_list()'.format(me,me))
    elif me == 'P+T':
        path = './{}/'.format(me)
        exec("df_PandT = pd.read_csv('{}new_best_vali_auc_{}.txt',sep='\t',header=None,engine='python')".format(path,me))
        exec('auc_PandT = df_PandT[1].to_list()')
    else:
        path = './{}/'.format(me)
        exec("df_{} = pd.read_csv('{}new_best_vali_auc_{}.txt',sep='\t',header=None,engine='python')".format(me,path,me))
        exec('auc_{} = df_{}[1].to_list()'.format(me,me))

auc = []
ap = []
name = []
row_length = 1 #The length of row is 1 here as we have 1 disease in one 'new_best_vali_auc*.txt' file.
for i in range(row_length):
    for me in methods:
        if me == 'PRScs':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            exec("df1_{} = pd.read_csv('./{}/new_score_PRScs_{}_{}.profile',sep=' +')".format(me,me,rep,para1))
            set1 = []
            for j in range(len(df1_PRScs.index)):
                if df1_PRScs.loc[j,'PHENO'] == -9:
                    set1.append(j)
            df1_PRScs.drop(set1,axis=0,inplace=True)
            exec("score_{} = df1_{}.loc[:,'SCORE'].to_list()".format(me,me))
            exec("df2_{} = pd.read_csv('./{}/new_test_score_PRScs_{}_{}.profile',sep=' +')".format(me,me,rep,para1))
            set2 = []
            for k in range(len(df2_PRScs.index)):
                if df2_PRScs.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_PRScs.drop(set2,axis=0,inplace=True)
            exec("test_score_{} = df2_{}.loc[:,'SCORE'].to_list()".format(me,me))
        elif me == 'LDpred2':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            exec("df1_{} = pd.read_csv('./{}/new_score_LDpred2_{}_{}.profile',sep=' +')".format(me,me,rep,para1))
            set1 = []
            for j in range(len(df1_LDpred2.index)):
                if df1_LDpred2.loc[j,'PHENO'] == -9:
                    set1.append(j)
            df1_LDpred2.drop(set1,axis=0,inplace=True)
            exec("score_{} = df1_{}.loc[:,'SCORE'].to_list()".format(me,me))
            exec("df2_{} = pd.read_csv('./{}/new_test_score_LDpred2_{}_{}.profile',sep=' +')".format(me,me,rep,para1))
            set2 = []
            for k in range(len(df2_LDpred2.index)):
                if df2_LDpred2.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_LDpred2.drop(set2,axis=0,inplace=True)
            exec("test_score_{} = df2_{}.loc[:,'SCORE'].to_list()".format(me,me))
        elif me == 'DBSLMM':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            para2 = para_set[2]
            exec("df1_{} = pd.read_csv('./{}/new_score_DBSLMM_{}_{}_{}.profile',sep=' +')".format(me,me,rep,para1,para2))
            set1 = []
            for j in range(len(df1_DBSLMM.index)):
                if df1_DBSLMM.loc[j,'PHENO'] == -9:
                    set1.append(j)
            df1_DBSLMM.drop(set1,axis=0,inplace=True)
            exec("score_{} = df1_{}.loc[:,'SCORE'].to_list()".format(me,me))
            exec("df2_{} = pd.read_csv('./{}/new_test_score_DBSLMM_{}_{}_{}.profile',sep=' +')".format(me,me,rep,para1,para2))
            set2 = []
            for k in range(len(df2_DBSLMM.index)):
                if df2_DBSLMM.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_DBSLMM.drop(set2,axis=0,inplace=True)
            exec("test_score_{} = df2_{}.loc[:,'SCORE'].to_list()".format(me,me))
        elif me == 'SBLUP':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            exec("df1_{} = pd.read_csv('./{}/new_score_SBLUP_{}_{}.profile',sep=' +')".format(me,me,rep,para1))
            set1 = []
            for j in range(len(df1_SBLUP.index)):
                if df1_SBLUP.loc[j,'PHENO'] == -9:
                    set1.append(j)
            df1_SBLUP.drop(set1,axis=0,inplace=True)
            exec("score_{} = df1_{}.loc[:,'SCORE'].to_list()".format(me,me))
            exec("df2_{} = pd.read_csv('./{}/new_test_score_SBLUP_{}_{}.profile',sep=' +')".format(me,me,rep,para1))
            set2 = []
            for k in range(len(df2_SBLUP.index)):
                if df2_SBLUP.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_SBLUP.drop(set2,axis=0,inplace=True)
            exec("test_score_{} = df2_{}.loc[:,'SCORE'].to_list()".format(me,me))
        elif me == 'SbayesR':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            para2 = para_set[2]
            exec("df1_{} = pd.read_csv('./{}/new_score_SbayesR_{}_{}_{}.profile',sep=' +')".format(me,me,rep,para1,para2))
            set1 = []
            for j in range(len(df1_SbayesR.index)):
                if df1_SbayesR.loc[j,'PHENO'] == -9:
                    set1.append(j)
            df1_SbayesR.drop(set1,axis=0,inplace=True)
            exec("score_{} = df1_{}.loc[:,'SCORE'].to_list()".format(me,me))
            exec("df2_{} = pd.read_csv('./{}/new_test_score_SbayesR_{}_{}_{}.profile',sep=' +')".format(me,me,rep,para1,para2))
            set2 = []
            for k in range(len(df2_SbayesR.index)):
                if df2_SbayesR.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_SbayesR.drop(set2,axis=0,inplace=True)
            exec("test_score_{} = df2_{}.loc[:,'SCORE'].values".format(me,me))
        elif me == 'P+T':
            exec("para_set = df_PandT.iloc[i,0].split('_')")
            rep = para_set[0]
            para1 = para_set[1]
            para2 = para_set[2]
            exec("df1_PandT = pd.read_csv('./{}/new_score_P+T_{}_pv{}_r{}.profile',sep=' +')".format(me,rep,para1,para2))
            set1 = []
            for j in range(len(df1_PandT.index)):
                if df1_PandT.loc[j,'PHENO'] == -9:
                    set1.append(j)
            df1_PandT.drop(set1,axis=0,inplace=True)
            exec("score_PandT = df1_PandT.loc[:,'SCORE'].to_list()")
            exec("df2_PandT = pd.read_csv('./{}/new_test_score_P+T_{}_{}_{}.profile',sep=' +')".format(me,rep,para1,para2))
            set2 = []
            for k in range(len(df2_PandT.index)):
                if df2_PandT.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_PandT.drop(set2,axis=0,inplace=True)
            exec("test_score_PandT = df2_PandT.loc[:,'SCORE'].to_list()")
        elif me == 'PRSice':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            para2 = para_set[2]
            exec("df1_{} = pd.read_csv('./{}/new_PRSice_{}_{}_{}.best',sep=' +')".format(me,me,rep,para1,para2))
            fam_PRSice = pd.read_csv('./{}_valid_new.fam'.format(rep),sep=' +',header=None)
            df1_PRSice['PHENO'] = fam_PRSice.iloc[:,5].values
            set1 = []
            for j in range(len(df1_PRSice.index)):
                if df1_PRSice.loc[j,'PHENO'] == -9:
                    set1.append(j)
            df1_PRSice.drop(set1,axis=0,inplace=True)
            exec("score_{} = df1_{}.loc[:,'PRS'].to_list()".format(me,me))
            exec("df2_{} = pd.read_csv('./{}/new_test_PRSice_{}_{}_{}.best',sep=' +')".format(me,me,rep,para1,para2))
            test_fam_PRSice = pd.read_csv('./{}_test_new.fam'.format(rep),sep=' +',header=None)
            df2_PRSice['PHENO'] = test_fam_PRSice.iloc[:,5].values
            set2 = []
            for k in range(len(df2_PRSice.index)):
                if df2_PRSice.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_PRSice.drop(set2,axis=0,inplace=True)
            exec("test_score_{} = df2_{}.loc[:,'PRS'].to_list()".format(me,me))
        elif me == 'PRSice2':
            exec("para_set = df_{}.iloc[i,0]".format(me))
            rep = para_set
            exec("df1_{} = pd.read_csv('./{}/new_PRSice2_{}.best',sep=' +')".format(me,me,rep))
            fam_PRSice2 = pd.read_csv('./{}_valid_new.fam'.format(rep),sep=' +',header=None)
            df1_PRSice2['PHENO'] = fam_PRSice2.iloc[:,5].values
            set1 = []
            for j in range(len(df1_PRSice2.index)):
                if df1_PRSice2.loc[j,'PHENO'] == -9:
                    set1.append(j)
            df1_PRSice2.drop(set1,axis=0,inplace=True)
            exec("score_{} = df1_{}.loc[:,'PRS'].to_list()".format(me,me))
            exec("df2_{} = pd.read_csv('./{}/new_test_PRSice2_{}.best',sep=' +')".format(me,me,rep))
            test_fam_PRSice2 = pd.read_csv('./{}_test_new.fam'.format(rep),sep=' +',header=None)
            df2_PRSice2['PHENO'] = test_fam_PRSice2.iloc[:,5].values
            set2 = []
            for k in range(len(df2_PRSice2.index)):
                if df2_PRSice2.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_PRSice2.drop(set2,axis=0,inplace=True)
            exec("test_score_{} = df2_{}.loc[:,'PRS'].to_list()".format(me,me))
        elif me == 'Tweedie':
            exec("para_set = df_{}.iloc[i,0]".format(me))
            rep = para_set
            exec("df1_{} = pd.read_csv('./{}/new_score_{}_{}.profile',sep=' +')".format(me,me,me,rep))
            set1 = []
            for j in range(len(df1_Tweedie.index)):
                if df1_Tweedie.loc[j,'PHENO'] == -9:
                    set1.append(j)
            df1_Tweedie.drop(set1,axis=0,inplace=True)
            valid_pheno = df1_Tweedie.loc[:,'PHENO'].to_list()
            exec("score_{} = df1_{}.loc[:,'SCORE'].to_list()".format(me,me))
            exec("df2_{} = pd.read_csv('./{}/new_test_score_{}_{}.profile',sep=' +')".format(me,me,me,rep))
            set2 = []
            for k in range(len(df2_Tweedie.index)):
                if df2_Tweedie.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_Tweedie.drop(set2,axis=0,inplace=True)
            test_pheno = df2_Tweedie.loc[:,'PHENO'].to_list()
            exec("test_score_{} = df2_{}.loc[:,'SCORE'].values".format(me,me))
        elif me == 'lassosum':
            exec("para_set = df_{}.iloc[i,0]".format(me))
            rep = para_set
            exec("df1_{} = pd.read_csv('./{}/new_score_{}_{}.profile',sep=' +')".format(me,me,me,rep))
            set1 = []
            for j in range(len(df1_lassosum.index)):
                if df1_lassosum.loc[j,'PHENO'] == -9:
                    set1.append(j)
            df1_lassosum.drop(set1,axis=0,inplace=True)
            valid_pheno = df1_lassosum.loc[:,'PHENO'].to_list()
            exec("score_{} = df1_{}.loc[:,'SCORE'].to_list()".format(me,me))
            exec("df2_{} = pd.read_csv('./{}/new_test_score_{}_{}.profile',sep=' +')".format(me,me,me,rep))
            set2 = []
            for k in range(len(df2_lassosum.index)):
                if df2_lassosum.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_lassosum.drop(set2,axis=0,inplace=True)
            test_pheno = df2_lassosum.loc[:,'PHENO'].to_list()
            exec("test_score_{} = df2_{}.loc[:,'SCORE'].values".format(me,me))
        elif me == 'EBPRS':
            exec("para_set = df_{}.iloc[i,0]".format(me))
            rep = para_set
            exec("df1_{} = pd.read_csv('./{}/new_score_{}_{}.profile',sep=' +')".format(me,me,me,rep))
            set1 = []
            for j in range(len(df1_EBPRS.index)):
                if df1_EBPRS.loc[j,'PHENO'] == -9:
                    set1.append(j)
            df1_EBPRS.drop(set1,axis=0,inplace=True)
            valid_pheno = df1_EBPRS.loc[:,'PHENO'].to_list()
            exec("score_{} = df1_{}.loc[:,'SCORE'].to_list()".format(me,me))
            exec("df2_{} = pd.read_csv('./{}/new_test_score_{}_{}.profile',sep=' +')".format(me,me,me,rep))
            set2 = []
            for k in range(len(df2_EBPRS.index)):
                if df2_EBPRS.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_EBPRS.drop(set2,axis=0,inplace=True)
            test_pheno = df2_EBPRS.loc[:,'PHENO'].to_list()
            exec("test_score_{} = df2_{}.loc[:,'SCORE'].values".format(me,me))
        elif me == 'tlpSum':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            para2 = para_set[2]
            exec("df1_{} = pd.read_csv('./penRegSum/new_score_{}_{}_{}_{}.profile',sep=' +')".format(me,me,rep,para1,para2))
            set1 = []
            for j in range(len(df1_tlpSum.index)):
                if df1_tlpSum.loc[j,'PHENO'] == -9:
                    set1.append(j)
            df1_tlpSum.drop(set1,axis=0,inplace=True)
            valid_pheno = df1_tlpSum.loc[:,'PHENO'].to_list()
            exec("score_{} = df1_{}.loc[:,'SCORE'].to_list()".format(me,me))
            exec("df2_{} = pd.read_csv('./penRegSum/new_test_score_{}_{}_{}_{}.profile',sep=' +')".format(me,me,rep,para1,para2))
            set2 = []
            for k in range(len(df2_tlpSum.index)):
                if df2_tlpSum.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_tlpSum.drop(set2,axis=0,inplace=True)
            test_pheno = df2_tlpSum.loc[:,'PHENO'].to_list()
            exec("test_score_{} = df2_{}.loc[:,'SCORE'].values".format(me,me))
        elif me == 'elastSum':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            para2 = para_set[2]
            exec("df1_{} = pd.read_csv('./penRegSum/new_score_{}_{}_{}_{}.profile',sep=' +')".format(me,me,rep,para1,para2))
            set1 = []
            for j in range(len(df1_elastSum.index)):
                if df1_elastSum.loc[j,'PHENO'] == -9:
                    set1.append(j)
            df1_elastSum.drop(set1,axis=0,inplace=True)
            valid_pheno = df1_elastSum.loc[:,'PHENO'].to_list()
            exec("score_{} = df1_{}.loc[:,'SCORE'].to_list()".format(me,me))
            exec("df2_{} = pd.read_csv('./penRegSum/new_test_score_{}_{}_{}_{}.profile',sep=' +')".format(me,me,rep,para1,para2))
            set2 = []
            for k in range(len(df2_elastSum.index)):
                if df2_elastSum.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_elastSum.drop(set2,axis=0,inplace=True)
            test_pheno = df2_elastSum.loc[:,'PHENO'].to_list()
            exec("test_score_{} = df2_{}.loc[:,'SCORE'].values".format(me,me))
    df_valid_score = np.array([score_PRScs,score_LDpred2,score_PandT,score_PRSice,score_DBSLMM,score_Tweedie,score_SBLUP,score_SbayesR,score_lassosum,score_EBPRS,score_tlpSum,score_elastSum,score_PRSice2])
    if df_valid_score.shape[1] != 13:
        df_valid_score = df_valid_score.T
    LR = LogisticRegression(max_iter=5000)
    valid_pheno = [int(x-1) for x in valid_pheno]
    LR.fit(df_valid_score,valid_pheno)
    df_test_score = np.array([test_score_PRScs,test_score_LDpred2,test_score_PandT,test_score_PRSice,test_score_DBSLMM,test_score_Tweedie,test_score_SBLUP,test_score_SbayesR,test_score_lassosum,test_score_EBPRS,test_score_tlpSum,test_score_elastSum,test_score_PRSice2])
    if df_test_score.shape[1] != 13:
        df_test_score = df_test_score.T
    Y = LR.predict_proba(df_test_score)
    test_pheno = [int(x-1) for x in test_pheno]
    auc.append(roc_auc_score(test_pheno,Y[:,1]))
    ap.append(average_precision_score(test_pheno,Y[:,1]))
    name.append('{}'.format(rep))
    df_score = pd.DataFrame(df2_elastSum.iloc[:,0].values)
    df_score[1] = Y[:,1]
    df_score[2] = test_pheno
    df_score.to_csv('new_test_predi_score_ensem.txt',sep='\t',header=False,index=False)
df = pd.DataFrame(name)
df[1] = auc
df.to_csv('new_test_auc_ensem.txt',sep='\t',header=False,index=False)
df1 = pd.DataFrame(name)
df1[1] = ap
df1.to_csv('new_test_ap_ensem.txt',sep='\t',header=False,index=False)

# PRS calculation

In [ ]:
import os
os.system('plink --bfile prefix_of_bed_file --score SNP_effect_sizes_file variant_IDs_column allele_codes_column SNP_effect_sizes_column --out output')

# AUROC and AUPRC evaluation

In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

name = []
auc = []
ap = []
pro = pd.read_csv('.profile file from PLINK',sep=' +')
set1 = []
for j in range(len(pro.index)):
    if pro.loc[j,'PHENO']==-9:
        set1.append(j)
pro.drop(set1,axis=0,inplace=True) #remove individuals of phenotypes being -9
y_true1 = pro.loc[:,'PHENO'].values
y_true = [abs(int(x)-1) for x in y_true1] #change disease label from 1 and 2 to 0 and 1
y_score = pro.loc[:,'SCORE'].values
name.append() #disease name and tuning parameters
auc.append(roc_auc_score(y_true,y_score))
ap.append(average_precision_score(y_true,y_score))
df = pd.DataFrame(name)
df[1] = auc
df[2] = ap
df.to_csv('results.txt',sep='\t',header=False,index=False)

# Odds ratio evaluation

In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import scipy.stats as stats
from sklearn import metrics
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm

methods = ['PRScs','LDpred2','P+T','PRSice','DBSLMM','Tweedie','SBLUP','SbayesR','PRSice2','lassosum','EBPRS','tlpSum','elastSum','ensem']
ML_methods = ['ada','GB','LR','NN','RF']

for me in methods:
    if me == 'tlpSum' or me == 'elastSum':
        path = './penRegSum/'
        exec("df_{} = pd.read_csv('{}new_best_vali_auc_{}.txt',sep='\t',header=None,engine='python')".format(me,path,me))
        exec('auc_{} = df_{}[1].to_list()'.format(me,me))
    elif me == 'P+T':
        path = './{}/'.format(me)
        exec("df_PandT = pd.read_csv('{}new_best_vali_auc_{}.txt',sep='\t',header=None,engine='python')".format(path,me))
        exec('auc_PandT = df_PandT[1].to_list()')
    elif me == 'ensem':
        pass
    else:
        path = './{}/'.format(me)
        exec("df_{} = pd.read_csv('{}new_best_vali_auc_{}.txt',sep='\t',header=None,engine='python')".format(me,path,me))
        exec('auc_{} = df_{}[1].to_list()'.format(me,me))


row_length = 1 #The length of row is 1 here as we have 1 disease in one 'new_best_vali_auc*.txt' file.
ratio = 0.01 #top ratio% versus the rest (1-ratio)%
for i in range(row_length):
    OR_set = []
    p_set = []
    confi_set = []
    name_set = []
    for me in methods:
        if me == 'PRScs':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            exec("df2_{} = pd.read_csv('./{}/new_test_score_PRScs_{}_{}.profile',sep=' +')".format(me,me,rep,para1))
            set2 = []
            for k in range(len(df2_PRScs.index)):
                if df2_PRScs.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_PRScs.drop(set2,axis=0,inplace=True)
            df2_PRScs.drop(['FID','IID','CNT','CNT2'],axis=1,inplace=True)
            df2_PRScs.sort_values(by='SCORE',ascending=False,inplace=True)
            row_num = len(df2_PRScs.index)
            row_place = int(row_num*ratio)
            top_1 = (df2_PRScs.PHENO[:row_place] == 1).sum()
            top_2 = (df2_PRScs.PHENO[:row_place] == 2).sum()
            rest_1 = (df2_PRScs.PHENO[row_place:] == 1).sum()
            rest_2 = (df2_PRScs.PHENO[row_place:] == 2).sum()
            ct = [[top_2,top_1],[rest_2,rest_1]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)
        elif me == 'LDpred2':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            exec("df2_{} = pd.read_csv('./{}/new_test_score_LDpred2_{}_{}.profile',sep=' +')".format(me,me,rep,para1))
            set2 = []
            for k in range(len(df2_LDpred2.index)):
                if df2_LDpred2.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_LDpred2.drop(set2,axis=0,inplace=True)
            df2_LDpred2.drop(['FID','IID','CNT','CNT2'],axis=1,inplace=True)
            df2_LDpred2.sort_values(by='SCORE',ascending=False,inplace=True)
            row_num = len(df2_LDpred2.index)
            row_place = int(row_num*ratio)
            top_1 = (df2_LDpred2.PHENO[:row_place] == 1).sum()
            top_2 = (df2_LDpred2.PHENO[:row_place] == 2).sum()
            rest_1 = (df2_LDpred2.PHENO[row_place:] == 1).sum()
            rest_2 = (df2_LDpred2.PHENO[row_place:] == 2).sum()
            ct = [[top_2,top_1],[rest_2,rest_1]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)
        elif me == 'DBSLMM':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            para2 = para_set[2]
            exec("df2_{} = pd.read_csv('./{}/new_test_score_DBSLMM_{}_{}_{}.profile',sep=' +')".format(me,me,rep,para1,para2))
            set2 = []
            for k in range(len(df2_DBSLMM.index)):
                if df2_DBSLMM.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_DBSLMM.drop(set2,axis=0,inplace=True)
            df2_DBSLMM.drop(['FID','IID','CNT','CNT2'],axis=1,inplace=True)
            df2_DBSLMM.sort_values(by='SCORE',ascending=False,inplace=True)
            row_num = len(df2_DBSLMM.index)
            row_place = int(row_num*ratio)
            top_1 = (df2_DBSLMM.PHENO[:row_place] == 1).sum()
            top_2 = (df2_DBSLMM.PHENO[:row_place] == 2).sum()
            rest_1 = (df2_DBSLMM.PHENO[row_place:] == 1).sum()
            rest_2 = (df2_DBSLMM.PHENO[row_place:] == 2).sum()
            ct = [[top_2,top_1],[rest_2,rest_1]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)
        elif me == 'SBLUP':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            exec("df2_{} = pd.read_csv('./{}/new_test_score_SBLUP_{}_{}.profile',sep=' +')".format(me,me,rep,para1))
            set2 = []
            for k in range(len(df2_SBLUP.index)):
                if df2_SBLUP.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_SBLUP.drop(set2,axis=0,inplace=True)
            df2_SBLUP.drop(['FID','IID','CNT','CNT2'],axis=1,inplace=True)
            df2_SBLUP.sort_values(by='SCORE',ascending=False,inplace=True)
            row_num = len(df2_SBLUP.index)
            row_place = int(row_num*ratio)
            top_1 = (df2_SBLUP.PHENO[:row_place] == 1).sum()
            top_2 = (df2_SBLUP.PHENO[:row_place] == 2).sum()
            rest_1 = (df2_SBLUP.PHENO[row_place:] == 1).sum()
            rest_2 = (df2_SBLUP.PHENO[row_place:] == 2).sum()
            ct = [[top_2,top_1],[rest_2,rest_1]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)
        elif me == 'SbayesR':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            para2 = para_set[2]
            exec("df2_{} = pd.read_csv('./{}/new_test_score_SbayesR_{}_{}_{}.profile',sep=' +')".format(me,me,rep,para1,para2))
            set2 = []
            for k in range(len(df2_SbayesR.index)):
                if df2_SbayesR.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_SbayesR.drop(set2,axis=0,inplace=True)
            df2_SbayesR.drop(['FID','IID','CNT','CNT2'],axis=1,inplace=True)
            df2_SbayesR.sort_values(by='SCORE',ascending=False,inplace=True)
            row_num = len(df2_SbayesR.index)
            row_place = int(row_num*ratio)
            top_1 = (df2_SbayesR.PHENO[:row_place] == 1).sum()
            top_2 = (df2_SbayesR.PHENO[:row_place] == 2).sum()
            rest_1 = (df2_SbayesR.PHENO[row_place:] == 1).sum()
            rest_2 = (df2_SbayesR.PHENO[row_place:] == 2).sum()
            ct = [[top_2,top_1],[rest_2,rest_1]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)
        elif me == 'P+T':
            exec("para_set = df_PandT.iloc[i,0].split('_')")
            rep = para_set[0]
            para1 = para_set[1]
            para2 = para_set[2]
            exec("df2_PandT = pd.read_csv('./{}/new_test_score_P+T_{}_{}_{}.profile',sep=' +')".format(me,rep,para1,para2))
            set2 = []
            for k in range(len(df2_PandT.index)):
                if df2_PandT.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_PandT.drop(set2,axis=0,inplace=True)
            df2_PandT.drop(['FID','IID','CNT','CNT2'],axis=1,inplace=True)
            df2_PandT.sort_values(by='SCORE',ascending=False,inplace=True)
            row_num = len(df2_PandT.index)
            row_place = int(row_num*ratio)
            top_1 = (df2_PandT.PHENO[:row_place] == 1).sum()
            top_2 = (df2_PandT.PHENO[:row_place] == 2).sum()
            rest_1 = (df2_PandT.PHENO[row_place:] == 1).sum()
            rest_2 = (df2_PandT.PHENO[row_place:] == 2).sum()
            ct = [[top_2,top_1],[rest_2,rest_1]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)
        elif me == 'PRSice':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            para2 = para_set[2]
            exec("df2_{} = pd.read_csv('./{}/new_test_PRSice_{}_{}_{}.best',sep=' +')".format(me,me,rep,para1,para2))
            test_fam_PRSice = pd.read_csv('./{}_test_new.fam'.format(rep),sep=' +',header=None)
            df2_PRSice['PHENO'] = test_fam_PRSice.iloc[:,5].values
            set2 = []
            for k in range(len(df2_PRSice.index)):
                if df2_PRSice.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_PRSice.drop(set2,axis=0,inplace=True)
            df2_PRSice.drop(['FID','IID','In_Regression'],axis=1,inplace=True)
            df2_PRSice.sort_values(by='PRS',ascending=False,inplace=True)
            row_num = len(df2_PRSice.index)
            row_place = int(row_num*ratio)
            top_1 = (df2_PRSice.PHENO[:row_place] == 1).sum()
            top_2 = (df2_PRSice.PHENO[:row_place] == 2).sum()
            rest_1 = (df2_PRSice.PHENO[row_place:] == 1).sum()
            rest_2 = (df2_PRSice.PHENO[row_place:] == 2).sum()
            ct = [[top_2,top_1],[rest_2,rest_1]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)
        elif me == 'PRSice2':
            exec("para_set = df_{}.iloc[i,0]".format(me))
            rep = para_set
            exec("df2_{} = pd.read_csv('./{}/new_test_PRSice2_{}.best',sep=' +')".format(me,me,rep))
            test_fam_PRSice2 = pd.read_csv('./{}_test_new.fam'.format(rep),sep=' +',header=None)
            df2_PRSice2['PHENO'] = test_fam_PRSice2.iloc[:,5].values
            set2 = []
            for k in range(len(df2_PRSice2.index)):
                if df2_PRSice2.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_PRSice2.drop(set2,axis=0,inplace=True)
            df2_PRSice2.drop(['FID','IID','In_Regression'],axis=1,inplace=True)
            df2_PRSice2.sort_values(by='PRS',ascending=False,inplace=True)
            row_num = len(df2_PRSice2.index)
            row_place = int(row_num*ratio)
            top_1 = (df2_PRSice2.PHENO[:row_place] == 1).sum()
            top_2 = (df2_PRSice2.PHENO[:row_place] == 2).sum()
            rest_1 = (df2_PRSice2.PHENO[row_place:] == 1).sum()
            rest_2 = (df2_PRSice2.PHENO[row_place:] == 2).sum()
            ct = [[top_2,top_1],[rest_2,rest_1]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)
        elif me == 'Tweedie':
            exec("para_set = df_{}.iloc[i,0]".format(me))
            rep = para_set
            exec("df2_{} = pd.read_csv('./{}/new_test_score_{}_{}.profile',sep=' +')".format(me,me,me,rep))
            set2 = []
            for k in range(len(df2_Tweedie.index)):
                if df2_Tweedie.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_Tweedie.drop(set2,axis=0,inplace=True)
            df2_Tweedie.drop(['FID','IID','CNT','CNT2'],axis=1,inplace=True)
            df2_Tweedie.sort_values(by='SCORE',ascending=False,inplace=True)
            row_num = len(df2_Tweedie.index)
            row_place = int(row_num*ratio)
            top_1 = (df2_Tweedie.PHENO[:row_place] == 1).sum()
            top_2 = (df2_Tweedie.PHENO[:row_place] == 2).sum()
            rest_1 = (df2_Tweedie.PHENO[row_place:] == 1).sum()
            rest_2 = (df2_Tweedie.PHENO[row_place:] == 2).sum()
            ct = [[top_2,top_1],[rest_2,rest_1]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)
        elif me == 'lassosum':
            exec("para_set = df_{}.iloc[i,0]".format(me))
            rep = para_set
            exec("df2_{} = pd.read_csv('./{}/new_test_score_{}_{}.profile',sep=' +')".format(me,me,me,rep))
            set2 = []
            for k in range(len(df2_lassosum.index)):
                if df2_lassosum.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_lassosum.drop(set2,axis=0,inplace=True)
            df2_lassosum.drop(['FID','IID','CNT','CNT2'],axis=1,inplace=True)
            df2_lassosum.sort_values(by='SCORE',ascending=False,inplace=True)
            row_num = len(df2_lassosum.index)
            row_place = int(row_num*ratio)
            top_1 = (df2_lassosum.PHENO[:row_place] == 1).sum()
            top_2 = (df2_lassosum.PHENO[:row_place] == 2).sum()
            rest_1 = (df2_lassosum.PHENO[row_place:] == 1).sum()
            rest_2 = (df2_lassosum.PHENO[row_place:] == 2).sum()
            ct = [[top_2,top_1],[rest_2,rest_1]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)
        elif me == 'EBPRS':
            exec("para_set = df_{}.iloc[i,0]".format(me))
            rep = para_set
            exec("df2_{} = pd.read_csv('./{}/new_test_score_{}_{}.profile',sep=' +')".format(me,me,me,rep))
            set2 = []
            for k in range(len(df2_EBPRS.index)):
                if df2_EBPRS.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_EBPRS.drop(set2,axis=0,inplace=True)
            df2_EBPRS.drop(['FID','IID','CNT','CNT2'],axis=1,inplace=True)
            df2_EBPRS.sort_values(by='SCORE',ascending=False,inplace=True)
            row_num = len(df2_EBPRS.index)
            row_place = int(row_num*ratio)
            top_1 = (df2_EBPRS.PHENO[:row_place] == 1).sum()
            top_2 = (df2_EBPRS.PHENO[:row_place] == 2).sum()
            rest_1 = (df2_EBPRS.PHENO[row_place:] == 1).sum()
            rest_2 = (df2_EBPRS.PHENO[row_place:] == 2).sum()
            ct = [[top_2,top_1],[rest_2,rest_1]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)
        elif me == 'tlpSum':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            para2 = para_set[2]
            exec("df2_{} = pd.read_csv('./penRegSum/new_test_score_{}_{}_{}_{}.profile',sep=' +')".format(me,me,rep,para1,para2))
            set2 = []
            for k in range(len(df2_tlpSum.index)):
                if df2_tlpSum.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_tlpSum.drop(set2,axis=0,inplace=True)
            df2_tlpSum.drop(['FID','IID','CNT','CNT2'],axis=1,inplace=True)
            df2_tlpSum.sort_values(by='SCORE',ascending=False,inplace=True)
            row_num = len(df2_tlpSum.index)
            row_place = int(row_num*ratio)
            top_1 = (df2_tlpSum.PHENO[:row_place] == 1).sum()
            top_2 = (df2_tlpSum.PHENO[:row_place] == 2).sum()
            rest_1 = (df2_tlpSum.PHENO[row_place:] == 1).sum()
            rest_2 = (df2_tlpSum.PHENO[row_place:] == 2).sum()
            ct = [[top_2,top_1],[rest_2,rest_1]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)
        elif me == 'elastSum':
            exec("para_set = df_{}.iloc[i,0].split('_')".format(me))
            rep = para_set[0]
            para1 = para_set[1]
            para2 = para_set[2]
            exec("df2_{} = pd.read_csv('./penRegSum/new_test_score_{}_{}_{}_{}.profile',sep=' +')".format(me,me,rep,para1,para2))
            set2 = []
            for k in range(len(df2_elastSum.index)):
                if df2_elastSum.loc[k,'PHENO'] == -9:
                    set2.append(k)
            df2_elastSum.drop(set2,axis=0,inplace=True)
            df2_elastSum.drop(['FID','IID','CNT','CNT2'],axis=1,inplace=True)
            df2_elastSum.sort_values(by='SCORE',ascending=False,inplace=True)
            row_num = len(df2_elastSum.index)
            row_place = int(row_num*ratio)
            top_1 = (df2_elastSum.PHENO[:row_place] == 1).sum()
            top_2 = (df2_elastSum.PHENO[:row_place] == 2).sum()
            rest_1 = (df2_elastSum.PHENO[row_place:] == 1).sum()
            rest_2 = (df2_elastSum.PHENO[row_place:] == 2).sum()
            ct = [[top_2,top_1],[rest_2,rest_1]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)
        elif me == 'ensem':
            df2_ensem = pd.read_csv('new_test_predi_score_ensem.txt',sep='\t',header=None)
            df2_ensem.columns = ['ID','SCORE','PHENO']
            df2_ensem.sort_values(by='SCORE',ascending=False,inplace=True)
            row_num = len(df2_ensem.index)
            row_place = int(row_num*ratio)
            top_0 = (df2_ensem.PHENO[:row_place] == 0).sum()
            top_1 = (df2_ensem.PHENO[:row_place] == 1).sum()
            rest_0 = (df2_ensem.PHENO[row_place:] == 0).sum()
            rest_1 = (df2_ensem.PHENO[row_place:] == 1).sum()
            ct = [[top_1,top_0],[rest_1,rest_0]]
            table = sm.stats.Table2x2(ct)
            OR = table.oddsratio
            confi = table.oddsratio_confint()
            p = table.oddsratio_pvalue()
            confi_set.append(confi)
            p_set.append(p)
            name_set.append('{}_{}'.format(rep,me))
            OR_set.append(OR)

    for ml in ML_methods:
        df = pd.read_csv('../ML/{}/test_predi_score_{}_{}.txt'.format(ml,rep,ml),sep=' ',header=None)
        df.columns = ['ID','SCORE','PHENO']
        df.sort_values(by='SCORE',ascending=False,inplace=True)
        row_num = len(df.index)
        row_place = int(row_num*ratio)
        top_0 = (df.PHENO[:row_place] == 0).sum()
        top_1 = (df.PHENO[:row_place] == 1).sum()
        rest_0 = (df.PHENO[row_place:] == 0).sum()
        rest_1 = (df.PHENO[row_place:] == 1).sum()
        ct = [[top_1,top_0],[rest_1,rest_0]]
        table = sm.stats.Table2x2(ct)
        OR = table.oddsratio
        confi = table.oddsratio_confint()
        p = table.oddsratio_pvalue()
        confi_set.append(confi)
        p_set.append(p)
        name_set.append('{}_{}'.format(rep,ml))
        OR_set.append(OR)
    df_tab = pd.DataFrame(name_set)
    df_tab[1] = OR_set
    df_tab[2] = confi_set
    df_tab[3] = p_set
    df_tab.to_csv('./new_OR_{}_ratio{}.txt'.format(rep,ratio),sep='\t',header=False,index=False)

# Pearson correlation coefficient

In [ ]:
import scipy.stats as stats
scores = # the scores of a method for target individuals
phenotypes = # the phenotypes of target individuals
pearson = stats.pearsonr(scores,phenotypes)

# Spearman's rank correlation coefficient

In [ ]:
import scipy.stats as stats
score1 = # the scores of method 1 for target individuals
score2 = # the scores of method 2 for target individuals
spearman = stats.spearmanr(score1,score1)